In [1]:
import databaseUtils.Request_builder as request_Builder
import databaseUtils.Utils as databaseUtils

In [5]:

context=dict()
context['input']="in1"
output=context['output']="ou1"
context['limit']=5
context['depth']=2
parameters = databaseUtils.Param(input=context['input'], output=context['output'], limit=context['limit'], depth=context['depth'])
builder = request_Builder.Requete(parameters)
cypher=builder.creer_Requete()
print(cypher)
"""
    builder.creer_Requete()
    utils = create_utils()
    result = utils.request(builder.requete)
    # print("La requête : ")
    # print(builder.requete)
    # print("Le résultat : ")
    # print(json.dumps(result, indent=2))
"""


            CALL {
                MATCH path=(i:IO {term: 'in1' })-[:inputOf|outputOf *1..2]->(o:IO {term: 'ou1' })
                // WHERE pour éviter de boucler sur une même fonction
                WHERE NONE (n IN nodes(path) WHERE size([x IN nodes(path) WHERE n = x]) > 1 )
                RETURN path
                LIMIT 5
                }
            //
            // pour l'instant un workflow est un unique chemin
            // mais est quand même représenté une liste de chemins (de 1 élément)
            WITH apoc.path.elements(path) AS pathAsList
            WITH [x in pathAsList | ID(x)] AS pathAsIdList
            WITH [pathAsIdList] AS workflow
            WITH collect(workflow) as workflowList
            
            CALL {
                WITH workflowList
                UNWIND workflowList AS pathList
                UNWIND pathList AS pathAsIdList
                UNWIND pathAsIdList AS stepID // pour chaque étape de workflow
                // on va chercher l'o